In [4]:
#Dependencies 
import numpy as np
import pandas as pd
import requests
import time
import json
from us import states
from scipy.stats import linregress
from matplotlib import pyplot as plt
from glob import glob


ModuleNotFoundError: No module named 'us'

In [5]:
##### Covid data from CDC API starting here:
url = "https://data.cdc.gov/resource/vbim-akqf.json"
print(requests.get(url))
print(requests.get(url).json())

url = "https://data.cdc.gov/resource/vbim-akqf.json"
response = requests.get(url)
response_json = response.json()
print(json.dumps(response_json, indent=4, sort_keys=True))

<Response [200]>
[{'cdc_report_dt': '2020-06-05T00:00:00.000', 'pos_spec_dt': '2020-04-08T00:00:00.000', 'current_status': 'Laboratory-confirmed case', 'sex': 'Male', 'age_group': '0 - 9 Years', 'race_ethnicity_combined': 'Black, Non-Hispanic', 'hosp_yn': 'Unknown', 'icu_yn': 'Unknown', 'death_yn': 'Unknown', 'medcond_yn': 'Unknown'}, {'cdc_report_dt': '2020-06-05T00:00:00.000', 'pos_spec_dt': '2020-04-08T00:00:00.000', 'onset_dt': '2020-04-08T00:00:00.000', 'current_status': 'Laboratory-confirmed case', 'sex': 'Male', 'age_group': '0 - 9 Years', 'race_ethnicity_combined': 'Black, Non-Hispanic', 'hosp_yn': 'No', 'icu_yn': 'Unknown', 'death_yn': 'No', 'medcond_yn': 'Unknown'}, {'cdc_report_dt': '2020-05-13T00:00:00.000', 'pos_spec_dt': '2020-04-08T00:00:00.000', 'onset_dt': '2020-05-03T00:00:00.000', 'current_status': 'Laboratory-confirmed case', 'sex': 'Male', 'age_group': '0 - 9 Years', 'race_ethnicity_combined': 'Black, Non-Hispanic', 'hosp_yn': 'Yes', 'icu_yn': 'Yes', 'death_yn': 'N

[
    {
        "age_group": "0 - 9 Years",
        "cdc_report_dt": "2020-06-05T00:00:00.000",
        "current_status": "Laboratory-confirmed case",
        "death_yn": "Unknown",
        "hosp_yn": "Unknown",
        "icu_yn": "Unknown",
        "medcond_yn": "Unknown",
        "pos_spec_dt": "2020-04-08T00:00:00.000",
        "race_ethnicity_combined": "Black, Non-Hispanic",
        "sex": "Male"
    },
    {
        "age_group": "0 - 9 Years",
        "cdc_report_dt": "2020-06-05T00:00:00.000",
        "current_status": "Laboratory-confirmed case",
        "death_yn": "No",
        "hosp_yn": "No",
        "icu_yn": "Unknown",
        "medcond_yn": "Unknown",
        "onset_dt": "2020-04-08T00:00:00.000",
        "pos_spec_dt": "2020-04-08T00:00:00.000",
        "race_ethnicity_combined": "Black, Non-Hispanic",
        "sex": "Male"
    },
    {
        "age_group": "0 - 9 Years",
        "cdc_report_dt": "2020-05-13T00:00:00.000",
        "current_status": "Laboratory-confirmed c

In [10]:
cases_df =  pd.DataFrame({
    "State": ["Georgia", "California", "Illinois", "Texas", "New York"],
    "Cases": [175052, 466550, 174973, 394265, 225850]})
cases_df

,State,Cases
0,Georgia,175052
1,California,466550
2,Illinois,174973
3,Texas,394265
4,New York,225850


In [14]:
##### Jayden and Max working on the airline data merge here:
from glob import glob

airport_merge = sorted(glob('/airlineCSV/TFMSC_Report_*.csv'))
airport_merge




[]

In [ ]:
########### Dave's code below for 2017-18 flu ################
# import and clean data from CDC FluView website csv download

In [ ]:
# read data file
raw_state_flu_2017 = pd.read_csv("../project_data/State_ILINet.csv")

In [ ]:
raw_state_flu_2017

In [ ]:
# collect a list of all columns within the DataFrame
raw_state_flu_2017.columns

In [ ]:
# purge unneeded columns
temp_state_flu_2017 = raw_state_flu_2017[['REGION', 'YEAR', 'WEEK', 'ILITOTAL']]
temp_state_flu_2017.head()

In [ ]:
# identify incomplete rows
temp_state_flu_2017.count()

In [ ]:
#check data types
temp_state_flu_2017.dtypes

In [ ]:
# drop rows with "X" in ILITOTAL column

# get names of indexes for which column ILITOTAL has value "X"
indexNames = temp_state_flu_2017[temp_state_flu_2017['ILITOTAL'] == 'X' ].index

# Delete these row indexes from dataFrame
temp_state_flu_2017.drop(indexNames , inplace=True)

In [ ]:
#check data types
temp_state_flu_2017.dtypes

In [ ]:
# use pd.to_numeric() method to convert the datatype of the ILITOTAL column
temp_state_flu_2017['ILITOTAL'] = pd.to_numeric(temp_state_flu_2017['ILITOTAL'])

In [ ]:
temp_state_flu_2017['ILITOTAL'].dtype

In [ ]:
# ******* NOT WORKING ******* NOT WORKING ******* NOT WORKING *********
# rename some columns     
clean_state_flu_2017 = temp_state_flu_2017.rename(columns={"State":"Year", "Week":"ILI Cases"})
clean_state_flu_2017

In [ ]:
# use GroupBy to separate the data into fields according to "State" values
#state_cases = clean_state_flu_2017.groupby(['ILITOTAL'])
#state_cases.sum().head()

# The object returned is a "GroupBy" object and cannot be viewed normally...
#print(state_cases)

# In order to be visualized, a data function must be used...
#state_cases.count().head(10)